In [1]:
using FFTW
using HDF5
using Plots
#using PyCall
using Random
using Measures
using Statistics
using LinearAlgebra
using LaTeXStrings

In [2]:
mutable struct Conf
    st::BitMatrix
end

#mutable struct Total_data
    #m_rho::Array{Float64, 3}
    #v_rho::Vector{Float64}
    #s_rho::Vector{Float64}
    #k_rho::Vector{Float64}
#end

In [3]:
function partition(a)
    part = []
    [append!(part, [[a[i,j], a[i,j+1], a[i+1,j], a[i+1,j+1]]]) for i in 1:2:size(a)[1] for j in 1:2:size(a)[2]]
    return part
end

partition (generic function with 1 method)

In [4]:
function combine(a, lx, ly)
    c = zeros(Bool, (lx,ly))
    p = 1
    for k in 1:convert(Int,lx/2)
        for j in 1:2:4
            com = []
            [append!(com, [a[i+((k-1)*convert(Int,ly/2))][j], a[i+((k-1)*convert(Int,ly/2))][j+1]]) for i in 1:convert(Int,ly/2)]
            c[p,:] = reduce(vcat,com)
            p += 1
        end
    end
    return c
end

combine (generic function with 1 method)

In [ ]:
function tetra_update!(tetramers, p)
    for j in eachindex(p)
        if (tetramers[j][1]==1 && tetramers[j][2]==0 && tetramers[j][3]==0 && tetramers[j][4]==0)
            @inbounds tetramers[j][1] = 0
            @inbounds tetramers[j][2] = p[j]
            @inbounds tetramers[j][3] = !p[j]
        elseif (tetramers[j][1]==0 && tetramers[j][2]==1 && tetramers[j][3]==0 && tetramers[j][4]==0)
            @inbounds tetramers[j][1] = p[j]
            @inbounds tetramers[j][2] = 0
            @inbounds tetramers[j][4] = !p[j]
        elseif (tetramers[j][1]==0 && tetramers[j][2]==0 && tetramers[j][3]==1 && tetramers[j][4]==0)
            @inbounds tetramers[j][1] = p[j]
            @inbounds tetramers[j][4] = !p[j]
            @inbounds tetramers[j][3] = 0
        elseif (tetramers[j][1]==0 && tetramers[j][2]==0 && tetramers[j][3]==0 && tetramers[j][4]==1)
            @inbounds tetramers[j][2] = p[j]
            @inbounds tetramers[j][3] = !p[j]
            @inbounds tetramers[j][4] = 0
        #=
        elseif (tetramers[j][1]==1 && tetramers[j][2]==1 && tetramers[j][3]==0 && tetramers[j][4]==0)
            @inbounds tetramers[j][1] = 0
            @inbounds tetramers[j][2] = 0
            @inbounds tetramers[j][3] = 1
            @inbounds tetramers[j][4] = 1
        elseif (tetramers[j][1]==1 && tetramers[j][2]==0 && tetramers[j][3]==1 && tetramers[j][4]==0)
            @inbounds tetramers[j][1] = 0
            @inbounds tetramers[j][2] = 1
            @inbounds tetramers[j][3] = 0
            @inbounds tetramers[j][4] = 1
        elseif (tetramers[j][1]==1 && tetramers[j][2]==0 && tetramers[j][3]==0 && tetramers[j][4]==1)
            @inbounds tetramers[j][1] = 0
            @inbounds tetramers[j][2] = 1
            @inbounds tetramers[j][3] = 1
            @inbounds tetramers[j][4] = 0
        elseif (tetramers[j][1]==0 && tetramers[j][2]==1 && tetramers[j][3]==0 && tetramers[j][4]==1)
            @inbounds tetramers[j][1] = 1
            @inbounds tetramers[j][2] = 0
            @inbounds tetramers[j][3] = 1
            @inbounds tetramers[j][4] = 0
        elseif (tetramers[j][1]==0 && tetramers[j][2]==0 && tetramers[j][3]==1 && tetramers[j][4]==1)
            @inbounds tetramers[j][1] = 1
            @inbounds tetramers[j][2] = 1
            @inbounds tetramers[j][3] = 0
            @inbounds tetramers[j][4] = 0
        elseif (tetramers[j][1]==0 && tetramers[j][2]==1 && tetramers[j][3]==1 && tetramers[j][4]==0)
            @inbounds tetramers[j][1] = 1
            @inbounds tetramers[j][2] = 0
            @inbounds tetramers[j][3] = 0
            @inbounds tetramers[j][4] = 1
            =#
        end
    end
    return
end

In [5]:
function tetra_update(tetramer)
    if tetramer == [1,0,0,0]
        return rand([[0,1,0,0], [0,0,1,0], [0,0,0,1]])
    elseif tetramer == [0,1,0,0]
        return rand([[1,0,0,0], [0,0,1,0], [0,0,0,1]])
    elseif tetramer == [0,0,1,0]
        return rand([[1,0,0,0],[ 0,1,0,0], [0,0,0,1]])
    elseif tetramer == [0,0,0,1]
        return rand([[1,0,0,0], [0,1,0,0], [0,0,1,0]])
    end
    return(tetramer)
end

tetra_update (generic function with 1 method)

In [6]:
function single_upd!(conf, lx, ly)
    off_x, off_y = rand(0:2), rand(0:2)
    #p = rand(Bool,convert(Int,lx*ly/4))
    conf.st = circshift(combine(map(tetra_update, partition(circshift(conf.st, (-off_x, -off_y)))), lx, ly), (off_x, off_y))
    return
end

single_upd! (generic function with 1 method)

In [7]:
function with_threads(samples, tmax, lx, ly)
    #rng = MersenneTwister()
    #Random.seed!(rng, Threads.threadid())
    n0 = 0.5
    #Ax, Ay = 0.1, 0.1
    A = 0.1
    qx, qy = convert(Int,floor(lx/10)), convert(Int,floor(ly/5))
    freq_x, freq_y = 2π * rfftfreq(lx), 2π * rfftfreq(ly)
    Qx, Qy = freq_x[qx], freq_y[qy]
    #A_x, A_y = cos.(Qx * (1:lx)), cos.(Qy * (1:ly))
    A_xy =  cos.(Qx * (1:lx) .+ Qy * (1:ly)')
    #rho_in = @. n0 + Ax * A_x + Ay *A_y'
    rho_in = @. n0 + A * A_xy
    #my_data = Total_data(zeros(Float64, tmax+1), zeros(Float64, tmax+1), zeros(Float64, tmax+1), zeros(Float64, tmax+1))
    my_data = zeros(Float64, lx, ly, tmax+1)
    for i in samples
        #Random.seed!(rng, i)
        my_conf = Conf(rand(Float64, (lx,ly)) .<= rho_in)
        #rho_0 = sum(my_conf.st .* A_x .* A_y')/(lx*ly)
        @inbounds my_data[:,:,1] .+= my_conf.st
        #@inbounds my_data.m_rho[:,:,1] .+= rho_0
        #@inbounds my_data.v_rho[1] += rho_0^2
        #@inbounds my_data.s_rho[1] += rho_0^3
        #@inbounds my_data.k_rho[1] += rho_0^4
        for t in 1:tmax
            single_upd!(my_conf, lx, ly)
            #rho_t = sum(my_conf.st .* Ax)/l
            @inbounds my_data[:,:,t+1] .+= my_conf.st
            #@inbounds my_data.m_rho[:,:,t+1] .+= rho_t
            #@inbounds my_data.v_rho[t+1] += rho_t^2
            #@inbounds my_data.s_rho[t+1] += rho_t^3
            #@inbounds my_data.k_rho[t+1] += rho_t^4
        end
    end
    return my_data
end

with_threads (generic function with 1 method)

In [8]:
function sum_multi_thread(samples, tmax, lx, ly)
    chunks = Iterators.partition(1:samples, samples ÷ Threads.nthreads())
    tasks = map(chunks) do chunk
        Threads.@spawn with_threads(chunk,tmax,lx, ly)
    end
    chunk_sums = fetch.(tasks)
    return sum(chunk_sums)
end

sum_multi_thread (generic function with 1 method)

In [9]:
Lx, Ly = 100,20

(100, 20)

In [10]:
samples = 100000
tmax = 400

400

In [11]:
y = sum_multi_thread(samples, tmax, Lx, Ly)

In [ ]:
heatmap(y[:,:,1]/samples)

In [ ]:
heatmap(y[:,:,200]/samples)

In [ ]:
heatmap(y[:,:,400]/samples)

In [ ]:
n0 = 0.5
Ax, Ay = 0.1, 0.1
qx, qy = convert(Int,floor(Lx/10)), convert(Int,floor(Ly/5))
freq_x, freq_y = 2π * rfftfreq(Lx), 2π * rfftfreq(Ly)
Qx, Qy = freq_x[qx], freq_y[qy]
A_x, A_y = cos.(Qx * (1:Lx)), cos.(Qy * (1:Ly))
rho_in = @. n0 + Ax * A_x + Ay *A_y'

In [ ]:
m_rho_ft = rfft(y, [1,2])/(Lx*Ly)/samples

In [ ]:
plot(abs.(m_rho_ft)[qx,1,:])
plot!(abs.(m_rho_ft)[1,qy,:])
plot!(abs.(m_rho_ft)[qx,qy,:])
plot!(abs.(m_rho_ft)[3,4,:])

In [ ]:
plot(abs.(m_rho_ft)[qx,qy,:])
plot!(abs.(m_rho_ft)[3,4,:])

In [ ]:
D = 0.07

In [ ]:
plot(log.(abs.(m_rho_ft[qx,1,:])/abs.(m_rho_ft)[qx,1,1]))
plot!(log.(abs.(m_rho_ft[1,qy,:])/abs.(m_rho_ft)[1,qy,1]))
#plot!(-D*Qx^2*(0:10), c=:black,linestyle=:dash, label = L"D = 0.04")
#plot!(-D*Qy^2*(0:10), c=:black,linestyle=:dash, label = L"D = 0.04")
#plot!((10:200),-sqrt.(D*Qx^2*(10:200)), c=:black,linestyle=:dash, label = L"slope = 4")
#plot!((10:400),-sqrt.(D*Qy^2*(10:400)).+0.6, c=:black,linestyle=:dash, label = L"slope = 4")

In [ ]:
plot(sqrt.(0:200),log.(abs.(m_rho_ft[qx,1,:])/abs.(m_rho_ft)[qx,1,1]))
plot!(sqrt.(0:200),log.(abs.(m_rho_ft[1,qy,:])/abs.(m_rho_ft)[1,qy,1]))

In [ ]:
h5open("../data/2D_Q10-5_mode_n0-0.5_-($samples)_tmax-($tmax)_lx-($Lx)_ly-($Ly).h5", "w") do file
    write(file, "m_rho", y)
end

In [ ]:
y = h5open("../data/2D_Q10-5_mode_n0-0.5_-(100000)_tmax-(400)_lx-(100)_ly-(20).h5", "r") do file
    read(file, "m_rho")
end

In [ ]:
map(tetra_update, partition())